### Oil type by origin
Goal: Determine oil type weighting in tank traffic by destination and vessel type


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
from monte_carlo_utils import decimal_divide

In [2]:
# User inputs
file_dir  = '/Users/rmueller/Data/MIDOSS/DeptOfEcology/'
file_name = 'MuellerTrans4-30-20.xlsx'

# This is the precision used to calculate oil type weights 
# Trial and error showed that a precision of 2 is neccessary for weights
# to sum to 1.0
precision = 2

In [3]:
# Import columns are: (G) Deliverer, (H) Receiver, (O) Region, (P) Product, 
#                     (Q) Quantity in Gallons, (R) Transfer Type (Fueling, Cargo, or Other)', 
#                     (w) DelivererTypeDescription, (x) ReceiverTypeDescription 
df = pd.read_excel(f'{file_dir}{file_name}',sheet_name='Vessel Oil Transfer', 
                   usecols="G,H,P,Q,R,W,X")

# create a dataframe for tug traffic from westridge
tugs_from_westridge = pd.DataFrame(columns = df.columns)

### create a list of all terminals included in the dataset, together with Region 
(so we can isolate Salish Sea) 

In [4]:
# The following list includes facilities used in Casey's origin/destination analysis 
# with names matching the Dept. of Ecology (DOE) database.  
# For example, the shapefile "Maxum Petroleum - Harbor Island Terminal" is labeled as 'Maxum (Rainer Petroleum)'
# in the DOE database.  I use the Ecology language here and will need to translate to Shapefile speak 
facility_names = [ 'BP Cherry Point Refinery', 'Shell Puget Sound Refinery', 'Shell Oil LP Seattle Distribution Terminal',
                   'Maxum (Rainer Petroleum)', 'Tidewater Snake River Terminal','Nustar Energy Tacoma', 
                   'SeaPort Sound Terminal', 'Tesoro Vancouver Terminal','Phillips 66 Ferndale Refinery', 
                   'Phillips 66 Tacoma Terminal', 'Andeavor Anacortes Refinery (formerly Tesoro)',
                   'Tesoro Port Angeles Terminal','U.S. Oil & Refining','US Oil Tacoma ','Naval Air Station Whidbey Island (NASWI)',
                   'NAVSUP Manchester', 'Alon Asphalt Company (Paramount Petroleum)', 'Kinder Morgan Liquids Terminal - Harbor Island',
                   'Nustar Energy Tacoma', 'Tesoro Pasco Terminal', 'REG Grays Harbor, LLC', 'Tidewater Vancouver Terminal',
                   'TLP','TLP Management Services LLC (TMS)']

## Create dictionary for fuel information

In [5]:
[nrows,ncols] = df.shape

# Create a destination dictionary
destination = {}
destination['names'] = facility_names.copy() 
destination['names'].remove('US Oil Tacoma ') # keeping 'U.S. Oil & Refining'
destination['names'].remove('TLP')           # 'TLP Management Services LLC (TMS)'

# Create an origin dictionary
origin = {}
origin['names'] = facility_names.copy()  #####  REMOVE TLP AND US OIL TACOMA FROM THIS LIST  ####
origin['names'].remove('US Oil Tacoma ') # keeping 'U.S. Oil & Refining'
origin['names'].remove('TLP')           # 'TLP Management Services LLC (TMS)'

# define vessel and fuel types to include in .yaml file
vessel_types = ['atb', 'barge', 'tanker']
fuel_type    = ['akns', 'bunker', 'dilbit', 'jet', 'diesel', 'gas', 'other']
data_type    = ['total_gallons', 'fraction_of_total', 'number_of_transfers']

for rows in range(nrows):
  
    # Create dictionary structure for destination
    if df.Receiver[rows] in facility_names:
        
        # create dictionary labels for each location (first two are special cases to catch bad data entry)       
        if (df.Receiver[rows] == 'TLP' or 
             df.Receiver[rows] == 'TLP Management Services LLC (TMS)'
           ):
            label = 'TLP Management Services LLC (TMS)'
        elif df.Receiver[rows] == 'U.S. Oil & Refining' or 'US Oil' in df.Receiver[rows]:
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Receiver[rows]}'
        
        # create dictionaries for each location     
        destination[label] = {}

        for vessel in vessel_types:
            destination[label][vessel] = {}
            for fuel in fuel_type:
                destination[label][vessel][fuel] = {}
                for data in data_type:
                    destination[label][vessel][fuel][f'{data}'] = 0

 # Create dictionary structure for origin
    if df.Deliverer[rows] in facility_names:
        
        # create dictionary labels for each location (first two are special cases to catch bad data entry)       
        if (df.Deliverer[rows] == 'TLP' or 
            df.Deliverer[rows] == 'TLP Management Services LLC (TMS)'
           ):
            label = 'TLP Management Services LLC (TMS)'      
        elif (df.Deliverer[rows] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Deliverer[rows]
             ):
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Deliverer[rows]}'
        
        # create dictionaries for each location     
        origin[label] = {}
        
        for vessel in vessel_types:
            origin[label][vessel] = {}
            for fuel in fuel_type:
                origin[label][vessel][fuel] = {}
                for data in data_type:
                    origin[label][vessel][fuel][f'{data}'] = 0


In [6]:
destination['names'] 

['BP Cherry Point Refinery',
 'Shell Puget Sound Refinery',
 'Shell Oil LP Seattle Distribution Terminal',
 'Maxum (Rainer Petroleum)',
 'Tidewater Snake River Terminal',
 'Nustar Energy Tacoma',
 'SeaPort Sound Terminal',
 'Tesoro Vancouver Terminal',
 'Phillips 66 Ferndale Refinery',
 'Phillips 66 Tacoma Terminal',
 'Andeavor Anacortes Refinery (formerly Tesoro)',
 'Tesoro Port Angeles Terminal',
 'U.S. Oil & Refining',
 'Naval Air Station Whidbey Island (NASWI)',
 'NAVSUP Manchester',
 'Alon Asphalt Company (Paramount Petroleum)',
 'Kinder Morgan Liquids Terminal - Harbor Island',
 'Nustar Energy Tacoma',
 'Tesoro Pasco Terminal',
 'REG Grays Harbor, LLC',
 'Tidewater Vancouver Terminal',
 'TLP Management Services LLC (TMS)']

## Loop through data again and catalog fuel information

In [7]:
## Loop through data again and catalog fuel information
unregulated_count=1
unregulated_gallons_in = 0
unregulated_gallons_out = 0
####  NOTE: I USE ROWS ABOVE AND ROW HERE.  THAT'S CRAZY MAKING.  I PLAN TO FIX. 
for row in range(nrows):
    
    # Catalogue fuel imports (destination traffic)
    if (df.Receiver[row] in facility_names and 
        df.TransferType[row] == 'Cargo'
       ):        
        if (df.Receiver[row] == 'TLP' or 
            df.Receiver[row] == 'TLP Management Services LLC (TMS)'
           ):            
            label = 'TLP Management Services LLC (TMS)'            
        elif (df.Receiver[row] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Receiver[row]
             ):            
            label = 'U.S. Oil & Refining'            
        else: # for the entries that are well-behaved!
            label = f'{df.Receiver[row]}'

        # Now ensure that the deliverer is a tank, barge or atb
        if df.DelivererTypeDescription[row] == 'TANK SHIP':

            # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[label]['tanker']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[label]['tanker']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[label]['tanker']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                destination[label]['tanker']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive of BIODIESEL, DIESEL/MARINE GAS OIL, and DIESEL LOW SULPHUR (ULSD)
                destination[label]['tanker']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['diesel']['number_of_transfers'] += 1
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                destination[label]['tanker']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['dilbit']['number_of_transfers'] += 1
            else:
                destination[label]['tanker']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['tanker']['other']['number_of_transfers'] += 1

        # ATBs
        elif ( df.DelivererTypeDescription[row] == 'TANK BARGE' or 
               df.DelivererTypeDescription[row] == 'TUGBOAT' and 
              'ATB' in df.Deliverer[row] or 
              'ITB' in df.Deliverer[row] ):           
            
             # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[label]['atb']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[label]['atb']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[label]['atb']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                destination[label]['atb']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['jet']['number_of_transfers'] += 1
                
            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                destination[label]['atb']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['diesel']['number_of_transfers'] += 1
                
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                destination[label]['atb']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['dilbit']['number_of_transfers'] += 1
                
            else:
                destination[label]['atb']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['atb']['other']['number_of_transfers'] += 1
        
        # Tank-barges
        elif ( df.DelivererTypeDescription[row] == 'TANK BARGE' or 
               df.DelivererTypeDescription[row] == 'TUGBOAT' and 
              'ATB' not in df.Deliverer[row] and 
              'ITB' not in df.Deliverer[row] ):
            
            # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                destination[label]['barge']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                destination[label]['barge']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                destination[label]['barge']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                destination[label]['barge']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                destination[label]['barge']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['diesel']['number_of_transfers'] += 1
            
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                destination[label]['barge']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['dilbit']['number_of_transfers'] += 1

            else:
                destination[label]['barge']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                destination[label]['barge']['other']['number_of_transfers'] += 1
            
            # find tugs transporting fuel from Westridge
            if label=='U.S. Oil & Refining':
                tugs_from_westridge = tugs_from_westridge.append( df.iloc[row] )
                
        else:
            unregulated_count += 1
            unregulated_gallons_in += df.TransferQtyInGallon[row].item()
            print(f'[{row}]: Cargo transfer of {df.TransferQtyInGallon[row].item():4.2e} gallons of {df.Product[row]} from {df.DelivererTypeDescription[row]} to {df.Receiver[row]} ')
            
   # Catalogue fuel exports (origin traffic)
    if (df.Deliverer[row] in facility_names and 
        df.TransferType[row] == 'Cargo'
       ):
        
        if (df.Deliverer[row] == 'TLP' or 
            df.Deliverer[row] == 'TLP Management Services LLC (TMS)'
           ):
            label = 'TLP Management Services LLC (TMS)'
        elif (df.Deliverer[row] == 'U.S. Oil & Refining' or 
              'US Oil' in df.Deliverer[row]
             ):
            label = 'U.S. Oil & Refining'
        else: # for the entries that are well-behaved!
            label = f'{df.Deliverer[row]}'

        # Now ensure that the deliverer is a tank, barge or atb
        if df.ReceiverTypeDescription[row] == 'TANK SHIP':

            # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[label]['tanker']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[label]['tanker']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[label]['tanker']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                origin[label]['tanker']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                origin[label]['tanker']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['diesel']['number_of_transfers'] += 1
             
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                origin[label]['tanker']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['dilbit']['number_of_transfers'] += 1

            else:
                origin[label]['tanker']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['tanker']['other']['number_of_transfers'] += 1
        
        # ATBs
        elif ( df.ReceiverTypeDescription[row] == 'TANK BARGE' or 
               df.ReceiverTypeDescription[row] == 'TUGBOAT' and 
              'ATB' in df.Receiver[row] or 
              'ITB' in df.Receiver[row] ):
             
             # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[label]['atb']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[label]['atb']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[label]['atb']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                origin[label]['atb']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIL, and DIESEL LOW SULPHUR (ULSD)
                origin[label]['atb']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['diesel']['number_of_transfers'] += 1
                
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                origin[label]['atb']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['dilbit']['number_of_transfers'] += 1

            else:
                origin[label]['atb']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['atb']['other']['number_of_transfers'] += 1
        
        # Tank-barges
        elif ( df.ReceiverTypeDescription[row] == 'TANK BARGE' or 
               df.ReceiverTypeDescription[row] == 'TUGBOAT' and 
              'ATB' not in df.Receiver[row] and 
              'ITB' not in df.Receiver[row] ):
            
            # Determine fuel-type in transfer 
            if 'CRUDE' in df.Product[row]:
                origin[label]['barge']['akns']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['akns']['number_of_transfers'] += 1
                
            elif df.Product[row] == 'BUNKER OIL/HFO':
                origin[label]['barge']['bunker']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['bunker']['number_of_transfers'] += 1

            elif df.Product[row] == 'GASOLINE':
                origin[label]['barge']['gas']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['gas']['number_of_transfers'] += 1

            elif df.Product[row] == 'JET FUEL/KEROSENE':
                origin[label]['barge']['jet']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['jet']['number_of_transfers'] += 1

            elif 'DIESEL' in df.Product[row]: 
                # inclusive if BIODIESEL, DIESEL/MARINE GAS OIl, and DIESEL LOW SULPHUR (ULSD)
                origin[label]['barge']['diesel']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['diesel']['number_of_transfers'] += 1
            
            elif 'BITUMEN OIL (DIL OR SYN)' in df.Product[row]: 
                origin[label]['barge']['dilbit']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['dilbit']['number_of_transfers'] += 1

            else:
                origin[label]['barge']['other']['total_gallons'] += df.TransferQtyInGallon[row].item()
                origin[label]['barge']['other']['number_of_transfers'] += 1
            
        else:
            unregulated_count += 1
            unregulated_gallons_out += df.TransferQtyInGallon[row].item()
            print(f'[{row}]: Cargo transfer of {df.TransferQtyInGallon[row].item():4.2e} gallons of {df.Product[row]} from {df.Deliverer[row]} to {df.ReceiverTypeDescription[row]} ')
            

[327]: Cargo transfer of 5.67e+05 gallons of JET FUEL/KEROSENE from Un-regulated Vessel to Naval Air Station Whidbey Island (NASWI) 
[658]: Cargo transfer of 1.05e+06 gallons of GASOLINE from BULK CARRIER to Tesoro Pasco Terminal 
[868]: Cargo transfer of 1.23e+07 gallons of DIESEL/MARINE GAS OIL from BP Cherry Point Refinery to Un-regulated Vessel 
[931]: Cargo transfer of 7.98e+05 gallons of GASOLINE from Un-regulated Vessel to Tesoro Pasco Terminal 
[932]: Cargo transfer of 1.01e+06 gallons of GASOLINE from Un-regulated Vessel to Tidewater Snake River Terminal 
[933]: Cargo transfer of 1.26e+06 gallons of ETHANOL from Tidewater Snake River Terminal to Un-regulated Vessel 
[965]: Cargo transfer of 5.67e+05 gallons of JET FUEL/KEROSENE from Un-regulated Vessel to Naval Air Station Whidbey Island (NASWI) 
[1125]: Cargo transfer of 3.28e+06 gallons of Z-OTHER from Un-regulated Vessel to REG Grays Harbor, LLC 
[1349]: Cargo transfer of 5.50e+04 gallons of DIESEL/MARINE GAS OIL from Maxum

In [8]:
unregulated_count

84

In [9]:
unregulated_gallons_out

72022739

In [10]:
unregulated_gallons_in

132401400

###  Set zero values to locations where no oil transfers are recorded for cargo

In [11]:
for facility in origin['names']:
    if facility not in origin:
        
        print(f'Origin: Adding null values to {facility}')
        
        # create dictionaries for each location     
        origin[facility] = {}
        
        for vessel in vessel_types:
            origin[facility][vessel] = {}
            for fuel in fuel_type:
                origin[facility][vessel][fuel] = {}
                for data in data_type:
                    origin[facility][vessel][fuel][f'{data}'] = 0

for facility in destination['names']:
    if facility not in destination:
        
        print(f'Origin: Adding null values to {facility}')
        
        # create dictionaries for each location     
        destination[facility] = {}
        
        for vessel in vessel_types:
            destination[facility][vessel] = {}
            for fuel in fuel_type:
                destination[facility][vessel][fuel] = {}
                for data in data_type:
                    destination[facility][vessel][fuel][f'{data}'] = 0


Origin: Adding null values to Shell Oil LP Seattle Distribution Terminal
Origin: Adding null values to Phillips 66 Tacoma Terminal
Origin: Adding null values to Naval Air Station Whidbey Island (NASWI)
Origin: Adding null values to REG Grays Harbor, LLC


In [12]:
## double check to make sure this is right.  

no_cargo_terminals = ['Shell Oil LP Seattle Distribution Terminal',
                     'Phillips 66 Tacoma Terminal',
                     'Naval Air Station Whidbey Island (NASWI)',
                     'REG Grays Harbor, LLC']
origin_check = {}
destination_check = {}
    
for terminal in no_cargo_terminals:
    df_destination = df[df['Receiver'] == terminal]
    df_origin = df[df['Deliverer'] == terminal]
    destination_check[terminal] = sum(
        df_destination[df_destination['TransferType'] == 'Cargo'].TransferQtyInGallon
    )
    origin_check[terminal] = sum(
        df_origin[df_origin['TransferType'] == 'Cargo'].TransferQtyInGallon
    )
    

In [13]:
destination_check

{'Shell Oil LP Seattle Distribution Terminal': 4242000,
 'Phillips 66 Tacoma Terminal': 94191800,
 'Naval Air Station Whidbey Island (NASWI)': 36749700,
 'REG Grays Harbor, LLC': 21094901}

In [14]:
origin_check

{'Shell Oil LP Seattle Distribution Terminal': 0,
 'Phillips 66 Tacoma Terminal': 0,
 'Naval Air Station Whidbey Island (NASWI)': 0,
 'REG Grays Harbor, LLC': 0}

### Loop through each location, vessel type and oil type and determine percent oil for each oil type

In [18]:
misbehaving_facility = []
misbehaving_vessel = []
for facility in origin['names']:
    for vessel in vessel_types:
        
        # first get the total transfers over all fuel types
        total_origin = sum(
            np.around(
                [origin[facility][vessel][count_me_in]['total_gallons'] for count_me_in in fuel_type]
            )
        )
        
        total_destination = sum(
            np.around(
                [destination[facility][vessel][count_me_in]['total_gallons'] for count_me_in in fuel_type]
        
            )
        )
    
        sum_origin = 0.0
        sum_destination = 0.0
        for fuel in fuel_type:
            # next calculate percentages
            if total_origin != 0:
                origin[facility][vessel][fuel]['fraction_of_total'] = decimal_divide(
                    origin[facility][vessel][fuel]['total_gallons'], 
                    total_origin,
                    precision
                )
                sum_origin += origin[facility][vessel][fuel]['fraction_of_total']
                    
#                 origin[facility][vessel][fuel]['fraction_of_total'] =  decimal_divide(
#                     origin[facility][vessel][fuel]['total_gallons'],
#                     total_origin,
#                     10
#                 )
                
            if total_destination != 0:
                destination[facility][vessel][fuel]['fraction_of_total'] = decimal_divide(
                    destination[facility][vessel][fuel]['total_gallons'],
                    total_destination,
                    precision
                )
                sum_destination += destination[facility][vessel][fuel]['fraction_of_total']
        
        if sum_origin != 0.0 and sum_origin != 1.0:
            misbehaving_facility.append(facility)
            misbehaving_vessel.append(vessel) 
            
            # manually force sum of weights to zero by adding or removing from "other"
            shortfall  = 1-sum_origin
            if shortfall != 1.0: # don't add oil transfers where none exist!
                if shortfall > 0.0: 
                    origin[facility][vessel]['other']['fraction_of_total'] = (
                        origin[facility][vessel]['other']['fraction_of_total'] + shortfall
                    )        
                    
        print(f'Sum of origin fuel weights: {sum_origin}')
        print(f'Sum of destination fuel weights: {sum_destination}')

Sum of origin fuel weights: 1.0
Sum of destination fuel weights: 1.01
Sum of origin fuel weights: 0.0
Sum of destination fuel weights: 0.0
Sum of origin fuel weights: 0.9999999999999999
Sum of destination fuel weights: 1.0
Sum of origin fuel weights: 1.01
Sum of destination fuel weights: 1.0
Sum of origin fuel weights: 0.0
Sum of destination fuel weights: 0.0
Sum of origin fuel weights: 1.0
Sum of destination fuel weights: 1.0
Sum of origin fuel weights: 0.0
Sum of destination fuel weights: 1.0
Sum of origin fuel weights: 0.0
Sum of destination fuel weights: 0.0
Sum of origin fuel weights: 0.0
Sum of destination fuel weights: 0.0
Sum of origin fuel weights: 1.0
Sum of destination fuel weights: 0.0
Sum of origin fuel weights: 0.0
Sum of destination fuel weights: 0.0
Sum of origin fuel weights: 1.0
Sum of destination fuel weights: 1.0
Sum of origin fuel weights: 1.0
Sum of destination fuel weights: 1.01
Sum of origin fuel weights: 1.0
Sum of destination fuel weights: 1.0
Sum of origin fu

In [19]:
misbehaving_facility

['BP Cherry Point Refinery',
 'Shell Puget Sound Refinery',
 'SeaPort Sound Terminal',
 'Andeavor Anacortes Refinery (formerly Tesoro)',
 'Kinder Morgan Liquids Terminal - Harbor Island']

In [20]:
misbehaving_vessel

['tanker', 'atb', 'atb', 'tanker', 'atb']

In [ ]:
# # Fix 'other' fraction of oil to be 1 - sum of other oil types
# for facility in origin['names']:
#     for vessel in vessel_types:
#         # Add all "non-other" fuel-types
#         origin_sum = sum([origin[facility][vessel][count_me_in]['fraction_of_total'] for count_me_in in fuel_type[0:6]])
#         destination_sum = sum([destination[facility][vessel][count_me_in]['fraction_of_total'] for count_me_in in fuel_type[0:6]])
        
#         # force sum of fraction_of_total to one by pooling "leftovers" in "other"       
#         if origin_sum > 1:
#             gas_correction = 1 - origin_sum
#             # gas works in this case but isn't a general fix
#             origin[facility][vessel]['gas']['fraction_of_total'] = (
#                 origin[facility][vessel]['gas']['fraction_of_total'] + gas_correction
#             )
#             print(f'{facility}, {vessel}, origin gas correction: {gas_correction}')
            
#         elif origin_sum < 1:
#             origin[facility][vessel]['other']['fraction_of_total'] =  (
#             1 - origin_sum
#         )
            
#         if destination_sum > 1:
#             gas_correction = 1 - destination_sum
#             origin[facility][vessel]['gas']['fraction_of_total'] = (
#                 origin[facility][vessel]['gas']['fraction_of_total'] + gas_correction
#             )
#             origin[facility][vessel]['other']['fraction_of_total'] = 0
#             print(f'{facility}, {vessel}, destination gas correction: {gas_correction}')          
        
#         elif destination_sum < 1:
#             destination[facility][vessel]['other']['fraction_of_total'] =  (
#             1 - destination_sum
#         )   

In [21]:
for fuel in fuel_type:
    string = str(destination['U.S. Oil & Refining']['atb'][fuel]['total_gallons'])
    print(string)
    
sum([destination['U.S. Oil & Refining']['atb'][fuel]['fraction_of_total'] for fuel in fuel_type])    

19950000
1776600
0
714000
19538400
9758994
4947600


0.99

In [22]:
for fuel in fuel_type:
    print(origin[facility][vessel][fuel]['fraction_of_total'])

0.0
0.0
0.0
0.0
1.0
0.0
0.0


In [23]:
for fuel in fuel_type:
    string = str(destination['U.S. Oil & Refining']['atb'][fuel]['fraction_of_total'])
    print(string)
    
sum([destination['U.S. Oil & Refining']['atb'][fuel]['fraction_of_total'] for fuel in fuel_type])    

0.35
0.03
0.0
0.01
0.34
0.17
0.09


0.99

In [24]:
for facility in origin['names']:
    for vessel in vessel_types:
        sum_check_out = [origin[facility][vessel][fuel]['fraction_of_total'] for fuel in fuel_type]
        sum_check_in  = [destination[facility][vessel][fuel]['fraction_of_total'] for fuel in fuel_type]
        if sum(sum_check_out) != 1:
            # add oil to "other" category to fix sum at 1
            shortfall  = 1-sum(sum_check_out)
            if shortfall != 1: # don't add oil transfers where none exist!
                origin[facility][vessel]['other']['fraction_of_total'] = (
                    origin[facility][vessel]['other']['fraction_of_total'] + shortfall
                )
                print(f'origin {vessel}, {facility} SHORTFALL:{shortfall}')

            # double check
            sum_check_out = [origin[facility][vessel][fuel]['fraction_of_total'] for fuel in fuel_type]
            if sum(sum_check_out) != 1 and sum(sum_check_out) !=0 :
                print(f'NOT THERE YET!: origin {vessel}, {facility}:{sum(sum_check_out)}')
        
        elif sum(sum_check_in) != 1:
            print(f'destination {vessel}, {facility}:{sum(sum_check_in)}')
            
            # add oil to "other" category to fix sum at 1
            shortfall  = 1-sum(sum_check_in)
            if shortfall != 1: # don't add oil transfers where none exist!
                destination[facility][vessel]['other']['fraction_of_total'] = (
                    destination[facility][vessel]['other']['fraction_of_total'] + shortfall
                )
                print(f'origin {vessel}, {facility} SHORTFALL:{shortfall}')

            # double check
            sum_check_in = [origin[facility][vessel][fuel]['fraction_of_total'] for fuel in fuel_type]
            if sum(sum_check_in) != 1 and sum(sum_check_in) !=0 :
                print(f'NOT THERE YET!: origin {vessel}, {facility}:{sum(sum_check_in)}')

destination atb, BP Cherry Point Refinery:1.01
origin atb, BP Cherry Point Refinery SHORTFALL:-0.010000000000000009
origin atb, Shell Puget Sound Refinery SHORTFALL:-0.010000000000000009
destination atb, Maxum (Rainer Petroleum):0
destination atb, Tidewater Snake River Terminal:1.01
origin atb, Tidewater Snake River Terminal SHORTFALL:-0.010000000000000009
origin atb, SeaPort Sound Terminal SHORTFALL:-0.010000000000000009
destination atb, Tesoro Vancouver Terminal:0
destination barge, Tesoro Vancouver Terminal:0
destination atb, Tesoro Port Angeles Terminal:0.99
origin atb, Tesoro Port Angeles Terminal SHORTFALL:0.010000000000000009
destination atb, U.S. Oil & Refining:0.99
origin atb, U.S. Oil & Refining SHORTFALL:0.010000000000000009
origin atb, Kinder Morgan Liquids Terminal - Harbor Island SHORTFALL:-0.010000000000000009
destination tanker, Kinder Morgan Liquids Terminal - Harbor Island:0
destination atb, TLP Management Services LLC (TMS):1.01
origin atb, TLP Management Services LL

In [25]:
sum([destination['U.S. Oil & Refining']['atb'][fuel]['fraction_of_total'] for fuel in fuel_type])

1.0

In [26]:
for fuel in fuel_type:
    string = str(destination['U.S. Oil & Refining']['atb'][fuel]['fraction_of_total'])
    print(string)
    destination['U.S. Oil & Refining']['atb'][fuel]['fraction_of_total'] = float(string)
    
sum([destination['U.S. Oil & Refining']['atb'][fuel]['fraction_of_total'] for fuel in fuel_type])    

0.35
0.03
0.0
0.01
0.34
0.17
0.1


1.0

In [30]:
origin


{'names': ['BP Cherry Point Refinery',
  'Shell Puget Sound Refinery',
  'Shell Oil LP Seattle Distribution Terminal',
  'Maxum (Rainer Petroleum)',
  'Tidewater Snake River Terminal',
  'Nustar Energy Tacoma',
  'SeaPort Sound Terminal',
  'Tesoro Vancouver Terminal',
  'Phillips 66 Ferndale Refinery',
  'Phillips 66 Tacoma Terminal',
  'Andeavor Anacortes Refinery (formerly Tesoro)',
  'Tesoro Port Angeles Terminal',
  'U.S. Oil & Refining',
  'Naval Air Station Whidbey Island (NASWI)',
  'NAVSUP Manchester',
  'Alon Asphalt Company (Paramount Petroleum)',
  'Kinder Morgan Liquids Terminal - Harbor Island',
  'Nustar Energy Tacoma',
  'Tesoro Pasco Terminal',
  'REG Grays Harbor, LLC',
  'Tidewater Vancouver Terminal',
  'TLP Management Services LLC (TMS)'],
 'Maxum (Rainer Petroleum)': {'atb': {'akns': {'total_gallons': 0,
    'fraction_of_total': 0.0,
    'number_of_transfers': 0},
   'bunker': {'total_gallons': 0,
    'fraction_of_total': 0.0,
    'number_of_transfers': 0},
   'di

In [29]:
with open(r'./WA_origin.yaml', 'w') as file:
    documents = yaml.safe_dump(origin, file)
with open(r'./WA_destination.yaml', 'w') as file:
    documents = yaml.safe_dump(destination, file)

RepresenterError: ('cannot represent an object', 0.0)

### plot up results for WA origin and destination by vessel type

In [ ]:
label = 'barge'
plot_origin = {}
facility = 'BP Cherry Point Refinery'
vessel_types = ['tanker','atb','barge']
for vessel in vessel_types:
    plot_origin[vessel] = np.empty(7)
    fuel_index = 0
    for fuel in fuel_type:
        plot_origin[vessel][fuel_index]=origin[facility][vessel][fuel]['total_gallons']
        fuel_index += 1

In [ ]:
# axes are in a two-dimensional array, indexed by [row, col]
plot_array = np.empty(len(fuel_type))

# Create a circle for the center of the plot
#my_circle=plt.Circle( (0,0), 0.7, color='white')

for i in range(len(destination['names'])):
    fig, axes = plt.subplots(1, len(vessel_types), figsize = (15,5))
    if i == 0:
        fig.suptitle('Washington fuel imports',
                    fontsize = 24)
    facility = origin['names'][i]
    plt.text(-.1, 0.5, 
             facility, 
             ha='left',
             fontsize=14,
             wrap=True,
             transform=plt.gcf().transFigure)
    for j in range(len(vessel_types)):
        vessel = vessel_types[j]
        # create array of total fuel volume by fuel type for each vessel at each facility
        fuel_index = 0
        for fuel in fuel_type:    
            plot_array[fuel_index]=destination[facility][vessel][fuel]['total_gallons']
            fuel_index += 1
        
        axes[j].axis('equal')
        
        if plot_array.sum()>0:
            axes[j].pie(plot_array, wedgeprops=dict(width=0.5), labels = fuel_type)
        else:
            axes[j].axis('off')
        axes[j].text(0,0,vessel,ha='center',fontsize=14)
       
          

In [ ]:
# axes are in a two-dimensional array, indexed by [row, col]
plot_array = np.empty(len(fuel_type))

# Create a circle for the center of the plot
#my_circle=plt.Circle( (0,0), 0.7, color='white')

for i in range(len(origin['names'])):
    fig, axes = plt.subplots(1, len(vessel_types), figsize = (15,5))
    if i == 0:
        fig.suptitle('Washington fuel exports',
                    fontsize = 24)
    facility = origin['names'][i]
    plt.text(-.1, 0.5, 
             facility, 
             ha='left',
             fontsize=14,
             wrap=True,
             transform=plt.gcf().transFigure)
    for j in range(len(vessel_types)):
        vessel = vessel_types[j]
        # create array of total fuel volume by fuel type for each vessel at each facility
        fuel_index = 0
        for fuel in fuel_type:    
            plot_array[fuel_index]=origin[facility][vessel][fuel]['total_gallons']
            fuel_index += 1
        
        axes[j].axis('equal')
        
        if plot_array.sum()>0:
            axes[j].pie(plot_array, wedgeprops=dict(width=0.5), labels = fuel_type)
        else:
            axes[j].axis('off')
        axes[j].text(0,0,vessel,ha='center',fontsize=14)
       
            
